# IMPLEMENTING TEMPORAL DIFFRENCE LEARNING IN FROZEN LAKE (openai's gym environment) using q-learning (qtable)

In [2]:
!python3 -m pip install gym

     |████████████████████████████████| 1.6 MB 13 kB/s 
     |████████████████████████████████| 1.0 MB 16 kB/s 
     |████████████████████████████████| 829 kB 26 kB/s 
  Created wheel for gym: filename=gym-0.17.2-py3-none-any.whl size=1650891 sha256=350afba66562fd108dbd1c13befabcc3a9f2c8a99ca410a9e5cf3dd42bf99bff
  Stored in directory: /Users/user/Library/Caches/pip/wheels/48/bf/7c/44b1b8e4ad998fc48e31caedbb9e028351861b8d20632642bc
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491058 sha256=5c6fba1fc38ecf96d8a8747a77d0425d09360bdcee8810a31c1981e715d463b2
  Stored in directory: /Users/user/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built gym future


## import packages gym for environment and performing trials, random for determining exploitation or exploration using episilon, and numpy for numerical computation in array-like manner.

In [1]:
import numpy as np
import gym
import random

## random example from https://gym.openai.com/docs/ to test my gym

In [12]:
env = gym.make('CartPole-v0')
env.reset()
for _ in range(1000):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

## define enironment for the agent

In [2]:
env = gym.make("FrozenLake-v0")

## get environment size for determining qtable. size is observable space and possible action is action space

In [3]:
action_size = env.action_space.n
state_size = env.observation_space.n

## initialize qtable

In [4]:
qtable = np.zeros((state_size, action_size))
print (qtable)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## defining our hyper parameters

In [5]:
total_episodes = 100000 #number of times the episode/ trials are done.
learning_rate = 0.7 #rate of update of action_space values in a state
max_steps = 99 #max_number of step to take in environment before end of episode
gamma = 0.95 #discount

## initialising learning episodial learning parameters

In [6]:
epsilon  = 1 # variable used in epsilon greedy algorithm
max_epsilon = 1 # the ceiling value of epsilon
min_epsilon = 0.01 # the floor value of epsilon 
epsilon_range = max_epsilon - min_epsilon # 
decay_rate = 0.005 # the power of e = 2.7182818285, natural number, by which we decrease epsilon

## initializing the agent's accumulated reward

In [7]:
accumulated_rewards = []
explore_exploit_data = []

## this block of code represents our agent performing actions in the environment 

In [8]:
for episode in range(total_episodes):
    state = env.reset() # we are starting from the left-uppermost position (i.e the beginning state of the frozen lake environment)
    total_reward = 0 # the reward our agent accumulates over the 100000 episodes
    dead = False
    for step in range(max_steps):
        num = random.random()
        if num > epsilon: # condition to determin exploration or exploitation
            action = np.argmax(qtable[state,:]) # exploitation
            explore_exploit_data.append("exploiting")
        else:
            action = env.action_space.sample()
            explore_exploit_data.append("exploring")
            # action = random.choice(qtable[state,:]) # exploration

        new_state, reward, dead, info = env.step(action) # getting the result of our action. a new state, a reward, information on our health, and general debugging info

        qtable[state,action] += learning_rate * (reward + gamma * max(qtable[new_state,:]) - qtable[state,action]) # the q-function for updating state value

        total_reward += reward # rewards gotten in the episode
        state = new_state # moving our agent to the newly acquired position or state

        if dead == True: # if our agent happened to die, stop the episode 
            break

    epsilon = min_epsilon + epsilon_range * np.exp(-decay_rate * episode) # carrying out exponential decay on epsilon to encourage less exploration as more episodes pass
    accumulated_rewards.append(total_reward) # keeps track of the reward accumulated

In [16]:
for i in range(0,len(accumulated_rewards),10000):
    print (accumulated_rewards[i])
print (sum(accumulated_rewards)) # to observe performance of agent for the hyperparameters we used.
print (qtable) # observe the value of state (the table can help tell us the state to be in a point in time and how to navigate the game to obtain maximium sccores)

1.0
5.0
[[3.15940148e-01 4.18405719e-02 4.19075565e-02 3.75728520e-02]
 [1.35689547e-02 1.21687754e-02 3.11019236e-02 3.98573907e-02]
 [1.51562674e-02 5.15704281e-02 2.24772804e-02 2.48552472e-02]
 [8.67740647e-03 1.87677661e-02 1.51824660e-02 3.66605297e-02]
 [4.97581287e-01 4.51365130e-02 2.14062876e-02 7.49869698e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.04709297e-03 5.06313211e-06 1.66656791e-02 6.58111640e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.96713310e-02 5.20557213e-01 2.50197900e-02 5.58682671e-01]
 [5.15343344e-02 7.62763507e-01 4.96532289e-02 1.13928675e-02]
 [2.20495454e-01 1.58280183e-02 1.17006524e-02 7.19475287e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.03689850e-01 5.98668397e-01 6.77280587e-01 6.98557600e-02]
 [1.84616156e-01 2.46293744e-01 8.98023110e-01 1.46883197e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 

## analysing exploit to explore ratio

In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(explore_exploit_data)

In [12]:
df.describe()

,0
count,3757279
unique,2
top,exploiting
freq,3717288


## agent exploited 31309 time while exploring 

## testing: navigating the frozen lake environment with the goal and start state set at 15 and 0 respectively

In [13]:
accumulated_rewards = [] # resetting our accumulate rewards
env.reset() # resetting our environment
for i in range(10): # performing 10 episodes using qtable
    state = env.reset()
    dead = False
    total_reward = 0
    for step in range(max_steps):
        action = np.argmax(qtable[state,:])

        new_state, reward, dead, info = env.step(action)

        total_reward += reward
        if new_state == 15: # if goal state is reached
            print ("eureka, agent has won!! in steps" + str(step+1) )
        if dead == True:
            env.render() # prints the environment if the agent dies
            break
        state = new_state

    accumulated_rewards.append(total_reward)

eureka, agent has won!! in steps15
  (Right)
SFFF
FHFH
FFFH
HFFG
eureka, agent has won!! in steps21
  (Right)
SFFF
FHFH
FFFH
HFFG
eureka, agent has won!! in steps48
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
eureka, agent has won!! in steps23
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
eureka, agent has won!! in steps17
  (Right)
SFFF
FHFH
FFFH
HFFG
